# Fine-tuning a Sequence Classification Model Exam

In this exam, you will be tasked with performing dataset preprocessing and fine-tuning a model for sequence classification. Complete each step carefully according to the instructions provided.

### Model and Dataset Information

For this task, you will be working with the following:

- **Model Checkpoint**: Use the pre-trained model checkpoint `aubmindlab/bert-base-arabertv02` for both the model and tokenizer.
- **Dataset**: You will be using the `CUTD/sanad_df` dataset. Ensure to load and preprocess the dataset correctly for training and evaluation.

**Note:**
- Any additional steps or methods you include that improve or enhance the results will be rewarded with bonus points if they are justified.
- The steps outlined here are suggestions. You are free to implement alternative methods or approaches to achieve the task, as long as you explain the reasoning and the process at the bottom of the notebook.
- You can use either TensorFlow or PyTorch for this task. If you prefer TensorFlow, feel free to use it when working with Hugging Face Transformers.
- The number of data samples you choose to work with is flexible. However, if you select a very low number of samples and the training time is too short, this could affect the evaluation of your work.

## Step 1: Load the Dataset

Load the dataset and split it into training and test sets. Use 20% of the data for testing.

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv02")
model = AutoModelForMaskedLM.from_pretrained("aubmindlab/bert-base-arabertv02")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/381 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/825k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.64M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02 were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
from datasets import load_dataset
data = load_dataset("CUTD/sanad_df")


sanad_df.csv:   0%|          | 0.00/50.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15000 [00:00<?, ? examples/s]

In [4]:
print(data)

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'label'],
        num_rows: 15000
    })
})


In [5]:
train_data=data['train']

In [6]:
data.set_format('pandas')
df = data['train'][:]
df.head(10)

,Unnamed: 0,text,label
0,0,الشارقة - محمد ولد محمد سالمعرضت مساء أمس الأو...,Culture
1,1,عبدالحكيم الزبيدي شاعر وقاص وناقد، جاءت نصوصه ...,Culture
2,2,انطلقت في مثل هذه الأيام من العام الفائت فعالي...,Culture
3,3,أقيمت مساء أمس الأول في إكسبو الشارقة ندوة حوا...,Culture
4,4,باسمة يونس حينما قال صاحب السموّ الشيخ الدكتور...,Culture
5,5,قرر اتحاد الأدباء والكتّاب الموريتانيين عقد مؤ...,Culture
6,6,تتناول الرواية الإماراتية المرأة ومكانتها في ا...,Culture
7,7,أعلنت هيئة دبي للثقافة والفنون (دبي للثقافة)، ...,Culture
8,8,توفي أمس الأديب الإسباني ميجيل ديليبس الذي يعت...,Culture
9,9,نظم مركز الشارقة للشعر الشعبي في مجلس ضاحية مغ...,Culture


## Step 2: Clean Unnecessary Columns

Remove any columns from the dataset that are not needed for training.

In [7]:
df.columns

Index(['Unnamed: 0', 'text', 'label'], dtype='object')

In [8]:
df = df[['text', 'label']]

In [9]:
df.head()

,text,label
0,الشارقة - محمد ولد محمد سالمعرضت مساء أمس الأو...,Culture
1,عبدالحكيم الزبيدي شاعر وقاص وناقد، جاءت نصوصه ...,Culture
2,انطلقت في مثل هذه الأيام من العام الفائت فعالي...,Culture
3,أقيمت مساء أمس الأول في إكسبو الشارقة ندوة حوا...,Culture
4,باسمة يونس حينما قال صاحب السموّ الشيخ الدكتور...,Culture


## Step 3: Splitting the Dataset

Split the dataset into training and testing sets, ensuring that 20% of the data is used for testing.

In [10]:
df.columns

Index(['text', 'label'], dtype='object')

In [11]:
from sklearn.model_selection import train_test_split



#X=df['text']
#y=df['label']

#X_train,X_test,y_train,y_test= train_test_split(X,y, test_size=0.2, random_state=42)



In [12]:
train_df , test_df=train_test_split(df ,test_size=0.2, random_state=42)

## Step 4: Tokenizing the Data

Initialize a tokenizer for the model.

In [13]:


from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv02")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

train_encodings = train_df.apply(lambda row: tokenize_function({"text": row["text"]}), axis=1)
test_encodings = test_df.apply(lambda row: tokenize_function({"text": row["text"]}), axis=1)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Step 5: Preprocessing the Text

Map the tokenization function to the dataset. Ensure the text data is processed using truncation to handle sequences that exceed the model's input size. Please do any further preprocessing.

**Bonus**: If you performed more comprehensive preprocessing, such as removing links, converting text to lowercase, or applying additional preprocessing techniques.

In [14]:
df.isnull().sum()

,0
text,0
label,0


In [15]:
df.duplicated().sum()

6

In [ ]:
df.drop_duplicates().sum()

In [ ]:
df.duplicated().sum()

### Step 6: Label Encoding

Convert the categorical labels into numerical format using a label encoder if needed.

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train_encoder = label_encoder.fit_transform(train_df['label'])
y_test_encoder = label_encoder.transform(test_df['label'])

### Step 7: Data Collation for Padding

Prepare the data for training by ensuring all sequences in a batch are padded to the same length. Use a data collator to handle dynamic padding.

In [ ]:

from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

### Step 8: Model Initialization

Initialize a sequence classification model using the BERT-based architecture. Set the the right amount of output labels.

In [ ]:
from transformers import AutoModelForMaskedLM

model = AutoModelForMaskedLM.from_pretrained("aubmindlab/bert-base-arabertv02")


## Step 9: Training Arguments

Define the training arguments, including parameters such as learning rate, batch size, number of epochs, and weight decay.

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv02")

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="no",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    num_train_epochs=1
)

## Step 10: Trainer Initialization and Training

Set up the trainer with the model, training arguments, dataset, tokenizer, and data collator. Train the model using the dataset you processed earlier.

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=X_train,
    eval_dataset=X_test,
    tokenizer=tokenizer,
    data_collator=data_collator
 )

In [ ]:
y_train_encoder

In [ ]:
X_train.shape , y_train.shape

In [ ]:
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

In [ ]:
trainer.train()

Once the model is trained, perform inference on a sample text to evaluate the model's prediction capabilities. Use the tokenizer to process the text, and then feed it into the model to get the predicted label.